In [ ]:
# the following code is used to load the preprocess data for model training
import numpy as np

train_label = np.load('data/train_label.npy')
train_categorical = np.load('data/train_categorical.npy') 
train_sparse = np.load('data/train_sparse.npy') 
train_numerical = np.load('data/train_numerical.npy')

test_label = np.load('data/test_label.npy')
test_categorical = np.load('data/test_categorical.npy') 
test_sparse = np.load ('data/test_sparse.npy') 
test_numerical = np. load ('data/test_numerical.npy')

In [ ]:
# the following code is used to set the random seed

import tensorflow as tf
import os
import numpy as np
import random

SEED = 179493511

def set_seeds(seed=SEED) :
    os.environ['PYTHONHASHSEED'] = str(seed) 
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds (seed=seed)

    os.environt['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1) 
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

In [ ]:
import os
import numpy as пр
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn import model_selection
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow_probability as tfp
import tqdm
from typing import Sequence

# install and import ltv
#!pip install -q git+https://github.com/google/lifetime_value 
import lifetime_value as ltv

In [ ]:
tfd = tfp.distributions
%config InlineBackend. figure_format='retina'
sns.set_style( 'whitegrid' )
pd.options.mode.chained_assignment = None # default= 'warn'

In [ ]:
from tensorflow.keras import initializers

In [ ]:
# the following two losses would be used in the model optimization stage

def mse(labels: tf.Tensor, logits: tf. Tensor) -› tf. Tensor:
    labels = labels[:, 0]

    label01 = tf.cast(tf.convert_to_tensor(labels, dtype=tf.float32) > 0, tf.float32)

    loss = tf.reduce_mean(label01 * tf.math.square(labels-logits[:,0]))*tf.reduce_sum(tf.ones_like(labels))/tf.reduce_sum(label01) 
    print(loss.shape)

    return loss

def cross(labels: tf.Tensor, logits: tf.Tensor) -> tf.Tensor:

    label01 = 1 - tf.cast(tf.convert_to_tensor(labels[:,0], dtype=tf.float32) > 0, tf.float32)

    return tf.reduce_mean(label01 * tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))*tf.reduce_sum(tf.ones_like(labels[:,0]))/tf.reduce_sum(label01)

In [ ]:
# the following code is used to load the pre-processed data, and these data are used to construct the collaborative filtlering feature

data_dir = '/home/ma-user/work/ltv_pytorch-master/raw_data/'
app_id_remap_dict = np.load(
    f'{data_dir}preprocess/app_id_remap_dict.npy', allow_pickle=True).item() # the app_id_remap_dict is used to count the number of all paid apps.
download_app_id_remap_dict = np.load(
    f'{data_dir}preprocess/download_app_id_remap_dict.npy', allow_pickle=True).item() # the download_app_id_remap_dict is used to count the number of all download apps

n_apps = max(app_id_remap_dict.values()) + 1
n_down_apps = max(download_app_id_remap_dict.values()) + 1

train_user_historical_downloads = np.load(
    f'{data_dir}preprocess/remap_train_download_app_ids.npy') 

train_app_id = np.load(f'{data_dir}preprocess/remap_train_app_ids.npy') 

train_user_historical_download_mask_1 = np.load(
    f' {data_dir}train_1/user._historical_downloads_mask.npy') 
train_user_historical_download_mask_2 = np.load(
    f'{data_dir}train_2/user_historical_downloads_mask.npy') 
train_user_historical_download_mask = np.concatenate(
    [train_user_historical_download_mask_1, train_user_historical_download_mask_2]) 

test_user_historical_downloads = \
    np.load(f'{data_dir})preprocess/remap_test_download_app_ids.npy') 
test_user_historical_download_mask = \
    np.load(f'{data_dir})test/user_historical_downloads_mask.npy') 
test_full_app_id = np.load(f'{data_dir}preprocess/test_full_app_id.npy')

In [ ]:
from tensorflow.keras. layers import Reshape, Embedding, Dense, Lambda, Input, Add, concatenate, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow. keras import backend as K
import tensorflow as tf

from tensorflow.keras import initializers, regularizers
tf. keras. backend.clear_session()
init = initializers.RandomUniform(minval=-0.01, maxval=0.01)

# the following code is used to model the linear relationship between the input features 

def featuresLinear(offset_dims, field_dims, x, input_mask):
    fc = tf.keras.layers.Embedding(field_dims, 1, embeddings_initializer=init)
    bias = tf.Variable(initial_value=tf.zeros((1,1)), trainabLe=True)
    offsets = np.array((0, *np.cumsum(offset_dims)), dtype=np.long)

    x = tf.cast(x, 'int32') + tf.expand_dims(tf.cast(offsets, 'int32'), axis=0)

    y = fc(x) * tf.reshape(input_mask, (-1, 11, 1)) + bias

    # <tf.Tensor 'add_1:0' shape=(None, 11, 1) dtype=float32>
    return Flatten()(y)

def featuresEmbedding(offset_mask, field_dims, n_factors, x, input_mask):
    offsets = np.array((0, *np.cumsum(offset_dims)), dtype=np.long)
    embedding = tf.keras.layers.Embedding(field_dims, n_factors, embeddings_initializer=init) 
    x = tf cast(x, 'int32') + tf.expand_dims(tf.cast(offsets, 'int32'), axis=0) 
    y = embedding(x) * tf.reshape(input_mask, (-1, 11, 1))

    return y

def fm(x, reduce_sum=False):
    square_of_sum = K.sum(x, axis=1) ** 2

    sum_of_square = K.sum(x ** 2, axis=1)
    ix = square_of_sum - sum_of_square 
    if reduce_sum:
        ix = K.sum(ix, axis=1, keepdims=True)
    return 0.5 * ix

In [ ]:
n_factors = 16
field_dims = n_down_apps + n_apps
offset_dims = [0] * 9
offset_dims.append(n_down_apps)

numeric_input = tf.keras.layers.Input(shape=(7,))
categorical_input = tf.keras.layers.Input(shape=(37,))
sparse_input = tf.keras.layers.Input(shape=(37,))

user_input = tf.keras.layers.Input(shape=(10,))
user_input_mask = tf.keras.layers.Input(shape=(10,))
game_input = tf.keras.layers.Input(shape=(1, ))

embedding_layer = tf.keras.layers.Embedding(22200, 16, embeddings_initializer=init) 
embedding_outputs = tf.keras.layers.Flatten()(embedding_layer(categorical_input))

game_input_mask = Lambda(Lambda X: K.ones_like(x))(game_input) 
input_mask = tf.keras.layers.concatenate([user_input_mask, game_input_mask])

x = tf.keras.layers.concatenate([user_input, game_input])

x1 = featuresLinear(offset_dims, field_dims, x, input_mask)
x2 = fm(featuresEmbedding(offset_dims, field_dims, n_factors, x, input_mask))
interaction_embedding = tf.keras.layers.concatenate([x1, x2]) # this interaction_embedding can be treated as the collaborative filtering feature

num_hidden = tf.keras.layers.BatchNormalization (axis=-1)(tf.keras.layers.Dense(64, activation='relu', kernel_initializer=init, bias_initializer=init) (numeric_input))

#import pdb; pdb.set_trace()

deep_input = tf.keras.layers.concatenate([num_hidden, interaction_embedding, embedding_outputs, sparse_input])

hidden_input = tf.keras.layers.BatchNormalization(axis=-1)(tf.keras.layers.Dense(256, activation='relu', kernel_initializer=init, bias_initializer=init)(deep_input))
hidden_input = tf.keras.layers.BatchNormalization (axis=-1)(tf.keras.layers.Dense(128, activation='relu', kernel_initializer=init, bias_initializer=init)(hidden_input))

out_1 = tf. keras. layers.Dense(3, kernel_initializer=init, bias_initializer=init)(hidden_input)
out_2 = tf.keras. layers.Dense(1, kernel_initializer=init, bias_initializer=init, activation='relu')(hidden_input) 
out_3 = tf.keras.layers.Dense(20, kernel_initializer=init, bias_initializer=init)(hidden_input)

final_out = tf.keras.layers.concatenate([out_1,out_2, out_3])

model = tf. keras.Model(inputs=[numeric_input, categorical_input, sparse_input,user_input, user_input_mask, game_input], outputs=[final_out, out_2, out_3])

In [ ]:
# the following loss function is inspired by the ZILN loss function: Wang et al., A Deep Probabilistic Model for Customer Lifetime Value Prediction. 
# it is very important in the spending money prediction task
def zero_inflated_gamma_loss(labels: tf.Tensor,
                                    logits: tf.Tensor) -> tf.Tensor:
    """Computes the zero inflated lognormal loss.

    Usage with tf.keras API:

    ```python
    model = tf. keras.Model(inputs, outputs)
    model.compile( 'sgd', loss=zero_inflated _ Lognormal)
    ```

    Arguments:
        Labels: True targets, tensor of shape [batch_size, 1]. 
        logits: Logits of output layer, tensor of shape [batch_size, 3].

    Returns:
        Zero inflated lognormal loss value.
    """

    labels = tf.convert_to_tensor(labels, dtype=tf.float32)
    labels = tf.squeeze(labels, axis=-1)

    label01 = tf.cast(tf.convert_to_tensor(labels, dtype=tf.float32) > 0, tf.float32)

    positive = tf.cast(labels > 0, tf.float32)

    logits = tf.convert_to_tensor(logits, dtypp=tf.float32)

    positive_logits = logits[..., 0]

    classification_loss = tf.keras.losses.binary_crossentropy(y_true=positive, y_pred=positive_logits, from_logits=True)

    alpha = tf.math.maximum(tf.nn.softplus(logits[:, 1]), 1e-4)
    beta = tf.math.maximum(tf.nn.softplus(logits[:, 2]), 1e-4)

    safe_labels = positive * labels + (
        1 - positive) * tf.keras.backend.ones_like(labels)
    regression_loss = -tf.keras.backend.mean (
        positive * tfd.Gamma(concentration=alpha, rate=beta).log_prob(safe_labels), axis=-1)

    label_reg = tf.math.log(1+ tf.math.maximum(labels, 0.))/tf.math.log(10.)
    contrast_01mask = tf.expand_dims(label01,axis=1)*tf.expand_dims(label01,axis=0)
    contrast_01negmask = tf.expand_dims(1-label01,axis=1)*tf.expand_dims(1-label01, axis=0)
    contrast_01crossmask = tf.expand_dims(label01,axis=1)*tf.expand_dims(1-label01, axis=0)#+tf.expand_dims(1-label01,axis=1)*tf.expand_dims(label01, axis=0)

    pred_miner = tf.expand_dims(logits[:,0],axis=1)-tf.expand_dims(logits[:,0],axis=0)

    cross_pred_miner = 2 * tf.reduce_mean(tf.reduce_mean(pred_miner * contrast_01crossmask, axis=-1),axis=-1, keepdims=True)*\ (tf.cast(tf.reduce_sum(tf.ones_like(labels),keepdims=True), tf.float32)**2/(tf.cast(tf.reduce_sum(tf.ones_Like(labels), keepdims=True), tf.float32) **2-tf.reduce_sum(label01, keepdims=True)**2-tf.reduce_sum(1-label01, keepdims=True)**2))

    pred_miner_norm = tf.expand_dims(tf.sigmoid(logits[:,0]),axis=1)-tf.expand_dims(tf.sigmoid(logits[:,0]),axis=0) 
    alpha = tf.math.maximum(tf.nn.softplus(logits[:, 1]), 1e-4)
    beta = tf.math.maximum(tf.nn.softplus(logits[:, 2]), 1e-4)

    predict_gamma = tf.math.log(1+(alpha/beta))/tf.math.log(10.)

    rec_reg = tf.math.log(tf.reduce_sum(tf.nn.softmax(logits[:, 4:])*(tf.cast(tf.ones_like(logits[:, 4:]), tf.float32)* tf.cast(2**tf.range(20), 
     tf.float32)),axis=-1))/tf.math.log (10.)
    predreg_miner3 = tf.expand_dims(logits[:,3],axis=1)-tf.expand_dims(logits[:,3],axis=0)
    predreg_miner = tf.expand_dims(tf.math.log(1+alpha/beta)/tf.math.log(10.),axis=1)-tf.expand_dims(tf.math.log(1+alpha/beta)/tf.math.log(10.),axis=0) cross_predreg_miner = tf.reduce_mean(tf.reduce_mean(tf.math.maximum(-predreg_miner*pred_miner_norm,0.)+tf.math.maximum (-predreg_miner3*pred_miner_norm,0.),axis=-1),axis=-1,keepdims=True)

    loss_reg_mse = cross_predreg_miner - tf.reduce_mean(tf.math.log(tf.math.sigmoid(cross_pred_miner)))#+ cross_predreg_miner

    print (cross_pred_miner.shape)
    return 2*classification loss + regression_loss + loss_reg_mse

loss = zero_inflated_gamma_loss#ltv.zero_inflated_lognormal_loss

model.compile(loss=[loss, mse, cross], optimizer=keras.optimizers.Adam(1r=2e-4),loss_weights=[1., 1., 0.5])

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint('weights.fepoch:02d}.hdf5', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)]

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss, r2_score, mean_squared_error, accuracy_score
from scipy.stats import pearson, spearmanr
from sklearn.metrics import mean_absolute_error
from tensorflow. keras.utils import to_categorical
all_pred = []

for i in range(5):
    pos_idx_list = np.where(cost_value_array>0)[0]
    neg_idx_List = random.sample(list(range(train_label.shape[0])), 4*pos_idx_list.shape[0])
    sample_idx_list = np.concatenate([pos_idx_list, neg_idx_List])

    model. fit([norm_train_numerical, train_categorical, train_sparse, train_user_historical_downloads, train_user_historical_download_mask, train_app_id], [train_label, np.expand_dims(np.log10(1+np.maximum(train_label,0.)),axis=1), to_categorical(np.floor(np.log2(1+np.maximum (train_label,0.))),20)], batch_size=10000,
        epochs=2,
        verbose=2, shuffle=True)#,callbacks=callbacks)
    
    logits, pred1, pred2 = model. predict([norm_test_numerical, test_categorical, test_sparse], batch_size=10000)

    pred0 = (tf.keras.backend.sigmoid(logits[:,0])*tf.keras.backend.softplus(logits[..., 1])/tf.keras.backend.softplus(logits[..., 2])). numpy()
    all_pred.append([logits, predi, pred2])
    prob = sigmoid(logits[:,:1])

    alpha = 0.3
    beta = 0.2
    preds = beta * pred0 + alpha*prob[:,0]*(10**pred1[:,0]-1) + (1-alpha- beta)*prob[:, 0]*(np.dot(scipy.special.softmax(pred2, axis=-1), 2**np.arange(20))-1)

    # 获取排序后的索引
    sorted_indices = sorted(range(len(preds)), key=lambda k: preds[k]) # 输出排序后的数据及其在原始列表中的索引 
    for idx, i in enumerate (sorted_indices):
        #print（collected_predictions［i］，"在原始列表中的位置：“，i）
        new_i = int(idx * len(ranking_cost_value_list) / len(sorted_indices)) 
        converted_predictions[i] = ranking_cost_value_list[new_i]

    x_origin_test_labels = []
    x_converted_predictions = []
    for idx, value in enumerate(origin_test_labels):
        if value > 0:
            x_origin_test_labels.append(value)
            x_converted_predictions.append(converted_predictions[idx])

    r2 = r2_score(x_origin_test_labels, x_converted_predictions) 
    print(f'r2_score: {"%.4f"%r2}')

    label, preds = test_label, preds

    rows = np.shape(label)[0]
    score_r2 = r2_score(label, preds)

    pred_norm1 = np.linalg.norm(x=preds, ord=2)
    true_norm = np.linalg.norm(x=label, ord=2)
    sim = np.dot(preds.reshape((1, rows)), label.reshape((rows, 1)))/pred_norm1/true_norm
    sim = sim[0, 0]
    not_zero_index = np.where(label != 0.)[0]
    score_r22 = r2_score(label[not_zero_index], preds[not_zero_index])

    regression_auc = roc_auo_score(np.array(label> 0.,dtype='int32'), preds)
    score_pearsonr_not_zero = pearsonr(label[not_zero_index], preds[not_zero_index].flatten())
    score_spearmanr_not_zero = spearmanr(label[not_zero_index], preds[not_zero_index].flatten())
    rmse = np.sqrt(mean_squared_error(label[not_zero_index], preds[not_zero_index].flatten()))
    mae = mean_absolute_error(label[not_zero_index], preds[not_zero_index].flatten())
    score_pearsonr = pearsonr(label, preds.flatten())
    score_spearmanr = spearmanr(label, preds.flatten())
    rmse2 = np.sqrt(mean_squared_error(label, preds.flatten()))
    mae2 = mean_absolute_error(label, preds.flatten() )

    # we adopt different kinds of metrics to evaluate the effectiveness of the proposed model, the score_r2 is the most important one. The smaller score_r2 denotes a better model. 
    print(score_r2, score_r22, score_pearsonr_not_zero[0], score_spearman_not_zero[0], score_pearsonr[0], score_spearmanr[0], rmse2, mae2, rmse, mse, regression_auc)

    # in ths following code, we test the perfomance of the model with different outputs. 
    for preds in [pred0, prob[:, 0]*(10**pred1[:,0]-1), prob[:, 0]*(np.dot(scipy.special.softmax(pred2, axis=-1), 2**np.arange(20))-1)]:
        rows = np.shape(label)[0]
        score_r2 = r2_score(label, preds)

        pred_norm1 = np.linalg.norm(x=preds, ord=2)
        true_norm = np.linalg.norm(x=label, ord=2)
        sim = np.dot(preds.reshape((1, rows)), label.reshape((rows, 1)))/pred_norm1/true_norm
        sim = sim[0, 0]
        not_zero_index = np.where(label != 0.)[0]
        score_r22 = r2_score(label[not_zero_index], preds[not_zero_index])

        regression_auc = roc_auc_score(np.array(Label> 0.,dtype='int32'), preds)
        score_pearsonr_not_zero = pearsonr(label([not_zeno_index], preds[not_zero_index].flatten())
        score_spearmanr_not_zero = spearmanr(label[not_zero_index], preds[not_zero_index].flatten())
        rmse = np.sqrt(mean_squared_error(label[not_zero_index], preds[not_zero_index]. flatten() ))
        mae = mean_absolute_error(label[not_zero_index], preds[not_zero_index].flatten() )
        score_pearsonr = pearsonr(label, preds.flatten())
        score_spearmanr = spearmanr(label, preds. flatten())
        rmse2 = np.sqrt(mean_squared_error(label, preds.flatten() ))
        mae2 = mean_absolute_error(label, preds.flatten())

        print(score_r2, score_r22, score_pearson_not_zero[0], score_spearmann_not_zero[0], score_pearson[0], score_spearmann[0], rmse2, mae2, rmse, mae, regression_auc)